# **The Problem & Business Importance**

> Problem
For sustainable growth, hotels need data-driven insights to predict customer churn and optimize room pricing.

Business Importance

> Business Importance



# **Importing Libraries & Reading Files**





In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from datetime import datetime
import os

from sklearn.model_selection import train_test_split

In [2]:
df_host = pd.read_csv('host_data.csv')
df_loc = pd.read_csv('location_data.csv')
df_pric = pd.read_csv('pricing_data.csv')
df_rev = pd.read_csv('review_data.csv')
df_prop = pd.read_csv('property_data.csv')

# **Data Identification & Analysis**


> Results of code below shows us interestingly that **'customer_id'** is present in all five datasets.




In [3]:
print(df_host.columns)
print(df_loc.columns)
print(df_pric.columns)
print(df_rev.columns)
print(df_prop.columns)

Index(['customer_id', 'host_since', 'host_response_rate',
       'host_has_profile_pic', 'host_identity_verified'],
      dtype='object')
Index(['customer_id', 'neighbourhood', 'latitude', 'longitude', 'city',
       'zipcode'],
      dtype='object')
Index(['customer_id', 'log_price', 'cleaning_fee', 'cancellation_policy'], dtype='object')
Index(['customer_id', 'number_of_reviews', 'review_scores_rating',
       'first_review', 'last_review'],
      dtype='object')
Index(['customer_id', 'property_type', 'room_type', 'bedrooms', 'bathrooms',
       'accommodates', 'beds'],
      dtype='object')




> This makes merging all datasets possible.




In [4]:
merge1 = pd.merge(df_host , df_loc , on='customer_id')
merge2 = pd.merge(merge1 , df_pric , on='customer_id')
merge3 = pd.merge(merge2 , df_prop , on='customer_id')
df = pd.merge(merge3 , df_rev , on='customer_id' )
#For now not merging the df_rev, due to incomplete values of the reviews.



> To get a overview and better understanding of data we do the following:


In [5]:
#Info on all columns of the merged dataset "df"
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110466 entries, 0 to 110465
Data columns (total 23 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   customer_id             110466 non-null  object 
 1   host_since              110083 non-null  object 
 2   host_response_rate      83463 non-null   object 
 3   host_has_profile_pic    110083 non-null  object 
 4   host_identity_verified  110083 non-null  object 
 5   neighbourhood           95456 non-null   object 
 6   latitude                110466 non-null  float64
 7   longitude               110466 non-null  float64
 8   city                    110466 non-null  object 
 9   zipcode                 109032 non-null  object 
 10  log_price               110466 non-null  float64
 11  cleaning_fee            110466 non-null  bool   
 12  cancellation_policy     110466 non-null  object 
 13  property_type           110466 non-null  object 
 14  room_type           

In [6]:
#Summary Statistics for all numerical values.
df.describe()


,latitude,longitude,log_price,bedrooms,bathrooms,accommodates,beds,number_of_reviews,review_scores_rating
count,110466.000000,110466.000000,110466.000000,110293.000000,110134.000000,110466.000000,110323.000000,110466.000000,85013.000000
mean,38.432660,-92.613310,4.645536,1.242445,1.233506,3.148073,1.716415,42.032734,94.026737
std,3.077177,21.726786,0.786136,0.872118,0.584256,2.184860,1.289476,75.730121,7.879345
min,33.338905,-122.511500,0.120000,-1.000000,0.000000,1.000000,0.000000,0.000000,20.000000
25%,34.127201,-118.345061,4.000000,1.000000,1.000000,2.000000,1.000000,2.000000,92.000000
50%,40.660739,-77.001637,4.715120,1.000000,1.000000,2.000000,1.000000,12.000000,96.000000
75%,40.746614,-73.954606,5.000000,1.000000,1.000000,4.000000,2.000000,46.000000,100.000000
max,42.390437,-70.985047,7.718399,10.000000,8.000000,16.000000,18.000000,1210.000000,100.000000


In [7]:
#All values of all columns of first 5 rows
df.head()

,customer_id,host_since,host_response_rate,host_has_profile_pic,host_identity_verified,neighbourhood,latitude,longitude,city,zipcode,...,property_type,room_type,bedrooms,bathrooms,accommodates,beds,number_of_reviews,review_scores_rating,first_review,last_review
0,4mUYtiM8BCkYchSi0rELebTJxPM68aZdKyUkER7ol5HI32...,10/07/2013,100%,t,t,Columbia Heights,38.921768,-77.030985,DC,20009,...,house,private room,1.0,1.0,2,1.0,0,NaN,NaN,NaN
1,XDBcz6zYzSF6AUGPCClRRhP6XjeHOHKCiybvgaevRF6J5Q...,07/10/2014,NaN,t,f,Westlake,34.057546,-118.288971,LA,90005,...,apartment,entire home/apt,0.0,1.0,2,1.0,0,NaN,NaN,NaN
2,OMm373CPtgtCt3dep7huwYt2G9uGUOPhEG7JkE8ZAvKlMp...,10/26/2016,100%,t,t,East New York,40.661924,-73.867641,NYC,11208.0,...,HOUSE,private room,1.0,1.0,2,1.0,26,91.0,2017-01-01,11/23/2018
3,OegBpWwCIpG3oc6jLqfYPpBRCCwGrcVh2xQsiUoZXSEtPp...,07/06/2016,NaN,t,t,South Pasadena,34.109652,-118.175605,LA,90042,...,villa,entire home/apt,1.0,1.0,3,1.0,0,NaN,NaN,NaN
4,837RAwLA6dEkJQHyGasNKdV3nZnunANTprjIo7J8C5wZL6...,10/03/2012,100%,t,f,Upper East Side,40.766836,-73.959440,NYC,10021,...,apartment,entire home/apt,1.0,1.0,2,1.0,64,98.0,2011-08-29,09/28/2018


In [8]:
#All values of all columns of last 5 rows
df.tail()

,customer_id,host_since,host_response_rate,host_has_profile_pic,host_identity_verified,neighbourhood,latitude,longitude,city,zipcode,...,property_type,room_type,bedrooms,bathrooms,accommodates,beds,number_of_reviews,review_scores_rating,first_review,last_review
110461,f2BOUClwcSJFylWFm9ZZbmT33NN5zfrEC9acLFniZftsMG...,11/22/2017,100%,t,t,Carroll Gardens,40.682925,-73.997469,NYC,11231.0,...,townhouse,ENTIRE HOME/APT,5.0,2.5,12,8.0,2,100.0,2017-08-06,10/06/2018
110462,owkLdHeTXuNky7id8h0OCRjocypRtUlBklyBzCTgt0qc57...,05/30/2016,60%,t,t,NaN,34.205908,-118.572788,LA,91306,...,house,PRIVATE ROOM,1.0,1.5,4,1.0,12,100.0,2015-10-11,03/30/2018
110463,rjooqKTDjUIHetuaCmvRgqsdCrwny9xJXw0aCMmB4xQUJp...,09/30/2015,97%,t,t,Bushwick,40.689701,-73.906398,NYC,11207,...,NONE,PRIVATE ROOM,1.0,1.0,2,1.0,0,NaN,NaN,NaN
110464,HuiGLAnSeU06GTAp8IL4DUWz1fAKG9n3aU6K46MdGBssGz...,11/21/2017,100%,t,f,Flatiron District,40.743522,-73.986369,NYC,10016,...,CONDOMINIUM,entire home/apt,2.0,1.0,6,3.0,14,94.0,2017-07-08,10/28/2018
110465,w7l65QhimlS0j0iOQkHzqXOx7zFWJJ5CHf7s5J7ueFP3Jc...,09/10/2010,88%,t,t,West End,42.364175,-71.063204,Boston,02114,...,apartment,entire home/apt,2.0,2.0,4,2.0,12,92.0,2012-05-29,07/28/2018


#Change the date

In [9]:
df['host_since'] = pd.to_datetime(df['host_since'])

# **Data Preparing, Cleaning and Visualization**


In [10]:
df.isna().sum()

customer_id                   0
host_since                  383
host_response_rate        27003
host_has_profile_pic        383
host_identity_verified      383
neighbourhood             15010
latitude                      0
longitude                     0
city                          0
zipcode                    1434
log_price                     0
cleaning_fee                  0
cancellation_policy           0
property_type                 0
room_type                     0
bedrooms                    173
bathrooms                   332
accommodates                  0
beds                        143
number_of_reviews             0
review_scores_rating      25453
first_review              24322
last_review               24286
dtype: int64

In [11]:
#Change customerid to int
df['customer_id'] = pd.factorize(df['customer_id'])[0] + 1
#host_response_rate to int
df['host_response_rate'] = pd.to_numeric(df['host_response_rate'].str.rstrip('%'))
#Change host_since to a numeric value ( we calculate the duration)
df['host_since'] = ((pd.to_datetime(datetime.now()) - df['host_since']).dt.days) / 365
#Change t/f to True and False
df['host_has_profile_pic'] = df['host_has_profile_pic'].map({'t': True, 'f': False})
df['host_identity_verified'] = df['host_identity_verified'].map({'t': True, 'f': False})
#Capatalize values in room_type and property_type to have a uniform standard between values
df['room_type'] = df['room_type'].str.upper()
df['property_type'] = df['property_type'].str.upper()


In [12]:
csv_file_path = 'data_FILLED_ZIPCODE_NEIGHBORS.csv'
if os.path.exists(csv_file_path):
    df = pd.read_csv(csv_file_path)
else:
    missing_values = df[df['zipcode'].isna()].index
    print(len(missing_values))
    for i in missing_values:
      row_data = df.loc[i]
      url = f"https://nominatim.openstreetmap.org/reverse?format=json&lat={row_data['latitude']}&lon={row_data['longitude']}"
      response = requests.get(url)
      print(response)
      data = response.json()
      df.loc[i, 'zipcode'] = data['address']['zipcode']


      missing_values_nb = df[df['neighbourhood'].isna()].index
      x = 0
      for i in missing_values_nb:
        print(f'{x} / {len(missing_values_nb)}')
        x += 1
        row_data = df.loc[i]
        url = f"https://nominatim.openstreetmap.org/reverse?format=json&lat={row_data['latitude']}&lon={row_data['longitude']}"
        response = requests.get(url)
        data = response.json()
        try:
          df.loc[i, 'neighbourhood'] = data['address']['neighbourhood']
        except:
          df.loc[i, 'neighbourhood'] = 'NO DATA AVAILABLE'

df.to_csv(csv_file_path, index=False)

1434


ConnectTimeout: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?format=json&lat=40.6612348483379&lon=-73.93428033386036 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000256A15364B0>, 'Connection to nominatim.openstreetmap.org timed out. (connect timeout=None)'))

In [ ]:
df.isna().sum()

In [ ]:
missing_values = df[(df['zipcode'].isna()) & (df['neighbourhood'].isna())]
missing_values

# Creating Dummies

In [ ]:
catgorical_columns = ['neighbourhood', 'city', 'cancellation_policy', 'property_type', 'room_type']
df_w_dummies = pd.get_dummies(df.drop(['customer_id','zipcode','latitude','longitude'], axis=1), columns=catgorical_columns, drop_first=True)
df_w_dummies.head()

In [ ]:
numeric_columns = ['host_since','host_response_rate','bedrooms', 'bathrooms','beds']


In [ ]:
for col in df_w_dummies.columns:
    df_w_dummies[col] = pd.to_numeric(df_w_dummies[col], errors='coerce')


To Do - Adnane
* clean value names seen in graphs below.

In [ ]:
# 1. Distribution of `log_price`
plt.figure(figsize=(10, 6))
sns.histplot(df['log_price'], bins=30, kde=True, color='blue', edgecolor='black')
plt.title('Distribution of Log Price', fontsize=16)
plt.xlabel('Log Price', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# 2. Countplot for `room_type`
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='room_type', palette='viridis')
plt.title('Room Type Distribution', fontsize=16)
plt.xlabel('Room Type', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.show()

# 3. Countplot for `property_type` (top 10 most common types)
plt.figure(figsize=(12, 6))
top_property_types = df['property_type'].value_counts().nlargest(10).index
sns.countplot(data=df[df['property_type'].isin(top_property_types)], y='property_type', palette='coolwarm', order=top_property_types)
plt.title('Top 10 Property Types', fontsize=16)
plt.xlabel('Count', fontsize=14)
plt.ylabel('Property Type', fontsize=14)
plt.show()

# 4. Boxplot for `accommodates` vs `log_price`
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='accommodates', y='log_price', palette='Spectral')
plt.title('Log Price vs Accommodates', fontsize=16)
plt.xlabel('Accommodates', fontsize=14)
plt.ylabel('Log Price', fontsize=14)
plt.show()

# 5. Countplot for `cancellation_policy`
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='cancellation_policy', palette='magma')
plt.title('Cancellation Policy Distribution', fontsize=16)
plt.xlabel('Cancellation Policy', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.show()

# 6. Scatterplot for `latitude` and `longitude` colored by `log_price`
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='longitude', y='latitude', hue='log_price', palette='viridis', alpha=0.6)
plt.title('Location of Listings (Colored by Log Price)', fontsize=16)
plt.xlabel('Longitude', fontsize=14)
plt.ylabel('Latitude', fontsize=14)
plt.legend(title='Log Price')
plt.show()

# 7. Barplot for `host_response_rate` (if numeric)
if df['host_response_rate'].dtype != object:
    plt.figure(figsize=(10, 6))
    sns.histplot(df['host_response_rate'], kde=True, color='orange', edgecolor='black')
    plt.title('Host Response Rate Distribution', fontsize=16)
    plt.xlabel('Host Response Rate (%)', fontsize=14)
    plt.ylabel('Frequency', fontsize=14)
    plt.show()

# 8. Boxplot for `bedrooms` vs `log_price`
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='bedrooms', y='log_price', palette='cool')
plt.title('Log Price vs Bedrooms', fontsize=16)
plt.xlabel('Bedrooms', fontsize=14)
plt.ylabel('Log Price', fontsize=14)
plt.show()

# 9. Boxplot for `bathrooms` vs `log_price`
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='bathrooms', y='log_price', palette='viridis')
plt.title('Log Price vs Bathrooms', fontsize=16)
plt.xlabel('Bathrooms', fontsize=14)
plt.ylabel('Log Price', fontsize=14)
plt.show()

# 10. Boxplot for `beds` vs `log_price`
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='beds', y='log_price', palette='plasma')
plt.title('Log Price vs Beds', fontsize=16)
plt.xlabel('Beds', fontsize=14)
plt.ylabel('Log Price', fontsize=14)
plt.show()

# 11. Countplot for `city`
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='city', palette='tab10', order=df['city'].value_counts().index)
plt.title('City Distribution', fontsize=16)
plt.xlabel('City', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.xticks(rotation=45)
plt.show()

# 12. Barplot for `neighbourhood` (Top 10 most common)
plt.figure(figsize=(12, 6))
top_neighbourhoods = df['neighbourhood'].value_counts().nlargest(10).index
sns.barplot(data=df[df['neighbourhood'].isin(top_neighbourhoods)],
            x='neighbourhood', y='log_price', estimator='mean', palette='Spectral', ci=None)
plt.title('Average Log Price by Top 10 Neighbourhoods', fontsize=16)
plt.xlabel('Neighbourhood', fontsize=14)
plt.ylabel('Average Log Price', fontsize=14)
plt.xticks(rotation=45)
plt.show()

# 13. Barplot for `cleaning_fee` vs `log_price`
plt.figure(figsize=(10, 6))
sns.barplot(data=df, x='cleaning_fee', y='log_price', palette='viridis', ci=None)
plt.title('Average Log Price by Cleaning Fee Presence', fontsize=16)
plt.xlabel('Cleaning Fee (True/False)', fontsize=14)
plt.ylabel('Average Log Price', fontsize=14)
plt.show()

# 14. Barplot for `host_identity_verified` vs `log_price`
plt.figure(figsize=(10, 6))
sns.barplot(data=df, x='host_identity_verified', y='log_price', palette='coolwarm', ci=None)
plt.title('Average Log Price by Host Identity Verification', fontsize=16)
plt.xlabel('Host Identity Verified (True/False)', fontsize=14)
plt.ylabel('Average Log Price', fontsize=14)
plt.show()

# 15. Barplot for `host_has_profile_pic` vs `log_price`
plt.figure(figsize=(10, 6))
sns.barplot(data=df, x='host_has_profile_pic', y='log_price', palette='Set2', ci=None)
plt.title('Average Log Price by Host Profile Picture Presence', fontsize=16)
plt.xlabel('Host Has Profile Picture (True/False)', fontsize=14)
plt.ylabel('Average Log Price', fontsize=14)
plt.show()


